# Segment Data from Poly detection

https://www.kaggle.com/competitions/bkai-igh-neopolyp/overview

## 1. Imports and Setup

#### 1.1 Imports

import all the required packages

In [2]:
import os
import glob
import pandas as pd

#### 1.2 Setup 

all the paths and where to find the data

In [3]:
data_src = r'C:\data\bkai-igh-neopolyp'
train_im_path = os.path.join(data_src, 'train')
masks_path = os.path.join(data_src, 'train_gt')
test_im_files = os.path.join(data_src, 'test')
csv_subm_data = os.path.join(data_src, 'sample_submission.csv')

## 2. Data (EDA, Preprocessing)

#### 2.1 Read data from disk

In [8]:
train_ims = glob.glob(os.path.join(train_im_path, '**/*.jpeg'), recursive=True)
masks = glob.glob(os.path.join(masks_path, '**/*.jpeg'), recursive=True)

In [7]:
print('found {0} train images and {1} mask images'.format(len(train_ims), len(masks)))

1000